In [1]:
!pip install -q --upgrade pip
!pip uninstall -y transformers huggingface_hub datasets
!pip install -q transformers==4.37.2 datasets==2.16.1 huggingface_hub==0.20.3 accelerate==0.27.2 peft==0.7.1


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 24.6 MB/s eta 0:00:0000:0100:01
Found existing installation: transformers 4.53.3
Uninstalling transformers-4.53.3:
  Successfully uninstalled transformers-4.53.3
Found existing installation: huggingface-hub 0.36.0
Uninstalling huggingface-hub-0.36.0:
  Successfully uninstalled huggingface-hub-0.36.0
Found existing installation: datasets 4.4.1
Uninstalling datasets-4.4.1:
  Successfully uninstalled datasets-4.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
pathos 0.3.2 requires dill>=0.3.8, but you have dill 0.3.7 which is incompatible.
pathos 0.3.2 requires multiprocess>=0.70.16, but you have multiprocess 0.70.15 which is incompatible.
s3fs 2025.3.0 requires fsspec==2025.3.0.*, but you have fsspec 2023.10.0 w

In [2]:
import transformers, datasets, huggingface_hub
print("transformers =", transformers.__version__)
print("datasets =", datasets.__version__)
print("hub =", huggingface_hub.__version__)


transformers = 4.37.2
datasets = 2.16.1
hub = 0.20.3


In [3]:
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [4]:
import transformers, datasets, huggingface_hub
print("transformers =", transformers.__version__)
print("datasets =", datasets.__version__)
print("hub =", huggingface_hub.__version__)


transformers = 4.37.2
datasets = 2.16.1
hub = 0.20.3


In [5]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [6]:
from datasets import load_dataset
from transformers import (
AutoTokenizer,
AutoModelForQuestionAnswering,
TrainingArguments,
Trainer,
default_data_collator
)
import numpy as np
import random
import collections

2025-11-09 03:01:45.229833: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762657305.418972      48 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762657305.471587      48 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [7]:
model_name = "deepset/deberta-v3-large-squad2"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)


# Freeze all parameters
for param in model.parameters():
    param.requires_grad = False

# Unfreeze the top 4 encoder layers
unfrozen_layers = ["layer.20", "layer.21", "layer.22", "layer.23"]  # DeBERTa has 24 layers (0-23)

for name, param in model.named_parameters():
    if any(layer_name in name for layer_name in unfrozen_layers):
        param.requires_grad = True

# Also unfreeze the QA head (output layer)
for name, param in model.qa_outputs.named_parameters():
    param.requires_grad = True


tokenizer_config.json:   0%|          | 0.00/392 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.65M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/18.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.74G [00:00<?, ?B/s]

In [8]:
trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
total = sum(p.numel() for p in model.parameters())
print(f"Trainable parameters: {trainable:,} / {total:,} ({100 * trainable / total:.2f}%)")


Trainable parameters: 50,386,946 / 434,014,210 (11.61%)


In [9]:
# Load a smaller subset for faster training
raw_datasets = load_dataset("hotpotqa/hotpot_qa", "fullwiki")
train_ds = raw_datasets["train"].shuffle(seed=42).select(range(45000))
val_ds = raw_datasets["validation"].shuffle(seed=42).select(range(5000))


max_length = 384
stride = 128

Generating train split:   0%|          | 0/90447 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/7405 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7405 [00:00<?, ? examples/s]

In [10]:
def prepare_train_features(examples):
    questions = examples["question"]
    contexts = [
        " ".join(sent for para in ctx["sentences"] for sent in para)
        for ctx in examples["context"]
    ]

    answers = [ans["text"] if isinstance(ans, dict) else ans for ans in examples["answer"]]
    
    
    tokenized = tokenizer(
        questions,
        contexts,
        truncation="only_second",
        max_length=max_length,
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )


    sample_mapping = tokenized.pop("overflow_to_sample_mapping")
    offset_mapping = tokenized.pop("offset_mapping")
    
    
    start_positions, end_positions = [], []
    for i, offsets in enumerate(offset_mapping):
        sample_idx = sample_mapping[i]
        answer = answers[sample_idx]
        context = contexts[sample_idx]
        cls_index = tokenized["input_ids"][i].index(tokenizer.cls_token_id)


        if not answer or not str(answer).strip():
            start_positions.append(cls_index)
            end_positions.append(cls_index)
            continue
        
        
        start_char = context.find(answer)
        if start_char == -1:
            start_positions.append(cls_index)
            end_positions.append(cls_index)
            continue
        
        
        end_char = start_char + len(answer)
        sequence_ids = tokenized.sequence_ids(i)
        
        
        token_start_index = 0
        while sequence_ids[token_start_index] != 1:
            token_start_index += 1


        token_end_index = len(sequence_ids) - 1
        while sequence_ids[token_end_index] != 1:
            token_end_index -= 1
        
        
        while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
            token_start_index += 1
        token_start_index -= 1
        
        
        token_end_index_tmp = token_start_index
        while token_end_index_tmp < len(offsets) and offsets[token_end_index_tmp][1] < end_char:
            token_end_index_tmp += 1
        token_end_index_tmp -= 1


        if token_start_index < 0 or token_end_index_tmp < 0:
            start_positions.append(cls_index)
            end_positions.append(cls_index)
        else:
            start_positions.append(token_start_index)
            end_positions.append(token_end_index_tmp)


    tokenized["start_positions"] = start_positions
    tokenized["end_positions"] = end_positions
    return tokenized

In [11]:
def prepare_validation_features(examples):
    questions = examples["question"]
    example_ids = examples["id"]
    contexts = [
        " ".join(sent for para in ctx["sentences"] for sent in para)
        for ctx in examples["context"]
    ]


    tokenized = tokenizer(
        questions,
        contexts,
        truncation="only_second",
        max_length=max_length,
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_mapping = tokenized.pop("overflow_to_sample_mapping")
    tokenized["example_id"] = [example_ids[sample_mapping[i]] for i in range(len(tokenized["input_ids"]))]

    # Dummy labels (required for evaluation to run)
    tokenized["start_positions"] = [0] * len(tokenized["input_ids"])
    tokenized["end_positions"] = [0] * len(tokenized["input_ids"])

    new_offsets = []
    for i, offsets in enumerate(tokenized["offset_mapping"]):
        sequence_ids = tokenized.sequence_ids(i)
        new_offsets.append([o if sequence_ids[k] == 1 else None for k, o in enumerate(offsets)])
    tokenized["offset_mapping"] = new_offsets

    return tokenized


In [12]:
train_dataset = train_ds.map(prepare_train_features, batched=True, remove_columns=train_ds.column_names, num_proc=2)
eval_dataset = val_ds.map(prepare_validation_features, batched=True, remove_columns=val_ds.column_names, num_proc=2)

Map (num_proc=2):   0%|          | 0/45000 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/5000 [00:00<?, ? examples/s]

In [13]:
# Only pass model-ready fields
def strip_eval(features):
    keep = ["input_ids", "attention_mask"]
    if "token_type_ids" in features.features:
        keep.append("token_type_ids")
    return features.remove_columns([col for col in features.column_names if col not in keep])

In [14]:
import numpy as np
import collections

def compute_hotpot_em_f1(pred_dict, references):
    def normalize_text(s):
        import re, string
        s = s.lower().strip()
        s = ''.join(ch for ch in s if ch not in string.punctuation)
        s = re.sub(r"\b(a|an|the)\b", " ", s)
        return ' '.join(s.split())

    def f1(pred, truth):
        pred_toks = normalize_text(pred).split()
        truth_toks = normalize_text(truth).split()
        common = collections.Counter(pred_toks) & collections.Counter(truth_toks)
        if not common:
            return 0.0
        prec = sum(common.values()) / len(pred_toks)
        rec = sum(common.values()) / len(truth_toks)
        return 2 * prec * rec / (prec + rec + 1e-12)

    id_to_ans = {ex["id"]: ex["answer"] for ex in references}
    em, f1_sum, total = 0, 0, 0
    for id_, pred in pred_dict.items():
        if id_ in id_to_ans:
            truth = id_to_ans[id_]
            em += normalize_text(pred) == normalize_text(truth)
            f1_sum += f1(pred, truth)
            total += 1
    return {"eval_em": em / total, "eval_f1": f1_sum / total} if total else {"eval_em": 0.0, "eval_f1": 0.0}


In [15]:
def postprocess_qa_predictions(examples, features, start_logits, end_logits):
    example_id_to_index = {k["id"]: i for i, k in enumerate(examples)}
    pred = {}
    features_per_example = collections.defaultdict(list)
    for i, f in enumerate(features):
        features_per_example[f["example_id"]].append(i)

    for ex_id, feat_ids in features_per_example.items():
        prelim = []
        for fid in feat_ids:
            start, end = start_logits[fid], end_logits[fid]
            offsets = features[fid]["offset_mapping"]
            s_ids = np.argsort(start)[-10:][::-1]
            e_ids = np.argsort(end)[-10:][::-1]
            for s in s_ids:
                for e in e_ids:
                    if s >= len(offsets) or e >= len(offsets): continue
                    if offsets[s] is None or offsets[e] is None: continue
                    if e < s or e - s + 1 > 30: continue
                    score = start[s] + end[e]
                    prelim.append((score, fid, s, e))
        if not prelim:
            pred[ex_id] = ""
            continue
        best = max(prelim, key=lambda x: x[0])
        _, fid, s, e = best
        offset = features[fid]["offset_mapping"]
        ctx = " ".join(sent for para in examples[example_id_to_index[ex_id]]["context"]["sentences"] for sent in para)
        pred[ex_id] = ctx[offset[s][0]:offset[e][1]] if offset[s] and offset[e] else ""
    return pred


In [16]:
def build_metrics():
    def fn(p):
        preds = postprocess_qa_predictions(
            val_ds,
            eval_dataset,
            np.array(p.predictions[0]),
            np.array(p.predictions[1])
        )
        scores = compute_hotpot_em_f1(preds, val_ds)

        # ✅ Include eval_loss directly if available
        if hasattr(p, "loss") and p.loss is not None:
            scores["eval_loss"] = p.loss

        return scores
    return fn


In [17]:
from transformers import TrainerCallback

class DebugLossLogger(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs:
            print(f"[Step {state.global_step}] ➤ {logs}")


In [30]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./debarta-squad-hotpot",
    evaluation_strategy="steps",
    eval_steps=3500,                     # Evaluate frequently (for test)
    logging_steps=500,                  # Log loss every 10 steps
    logging_first_step=True,          # Log step 1
    save_steps=7000,
    save_total_limit=1,

    num_train_epochs=1,               # Or 1, if short run
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=8,

    learning_rate=5e-6,
    fp16=True,
    dataloader_num_workers=2,
    dataloader_pin_memory=True,

    load_best_model_at_end=True,
    metric_for_best_model="eval_f1",  # matches your metric dict
    greater_is_better=True,           # for f1 metric

    logging_dir="./logs",
    log_level="info",
    report_to="none",                 # Avoids WandB or TensorBoard setup issues

    disable_tqdm=False                # Show training progress bar
)


PyTorch: setting up devices


In [31]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=default_data_collator,
    compute_metrics=build_metrics(),
    callbacks=[DebugLossLogger()] 
)

Using auto half precision backend


In [32]:
trainer.train()

***** Running training *****
  Num examples = 222,115
  Num Epochs = 1
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 8
  Total optimization steps = 13,882
  Number of trainable parameters = 50,386,946


Step,Training Loss,Validation Loss


[Step 1] ➤ {'loss': 1.6796, 'learning_rate': 4.999639821351391e-06, 'epoch': 0.0}


KeyboardInterrupt: 

In [ ]:
trainer.save_model("./deberta-v3-large-squad-v2-hotpot")

In [33]:
!rm -rf ./debarta-squad-hotpot

In [25]:
!zip -r deberta_fine_tuned_model.zip /kaggle/working/deberta-v3-large-squad-v2-hotpot

	zip warning: name not matched: /kaggle/working/deberta-v3-large-squad-v2-hotpot

zip error: Nothing to do! (try: zip -r deberta_fine_tuned_model.zip . -i /kaggle/working/deberta-v3-large-squad-v2-hotpot)
